In [3]:
# !pip install -U langchain-community
# !pip install sentence-transformers
# !pip install faiss-cpu
# !pip install --upgrade langchain
# !pip install fitz
# !pip install PyMuPDF
# !pip install groq

In [48]:
# Install packages
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

import fitz
from PIL import Image
import io
import json
import pandas as pd

from groq import Groq

In [49]:
# Functions

def extract_text(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text("text")
    return text

def split_documents_into_chunks(docs, chunk_size=500, chunk_overlap=100):
    """
    Splits the given documents into chunks of specified size with overlap.

    Args:
        docs (list): List of documents to split.
        chunk_size (int): Size of each chunk. Default is 500 characters.
        chunk_overlap (int): Overlap size between chunks. Default is 100 characters.

    Returns:
        list: List of split documents with chunks.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    return text_splitter.split_documents(docs)

def add_chunk_numbers_to_metadata(doc_splits):
    """
    Adds chunk numbers to the metadata of each split document.

    Args:
        doc_splits (list): List of split documents.

    Returns:
        list: List of split documents with updated metadata.
    """
    for idx, split in enumerate(doc_splits):
        split.metadata["chunk"] = idx
    return doc_splits

def configure_faiss_vector_store(doc_splits, embeddings):
    """
    Configures FAISS as the vector store using the provided document splits and embeddings.

    Args:
        doc_splits (list): List of split documents.
        embeddings (Embeddings): Embeddings to be used for FAISS.

    Returns:
        FAISS: Configured FAISS vector store.
    """
    return FAISS.from_documents(doc_splits, embeddings)

def create_retriever(vector_store, search_type="similarity", k=5):
    """
    Exposes the vector store index to a retriever.

    Args:
        vector_store: The vector store (e.g., FAISS, Annoy, etc.).
        search_type (str): The type of search to perform. Default is "similarity".
        k (int): The number of documents to return. Default is 5.

    Returns:
        retriever: The configured retriever.
    """
    return vector_store.as_retriever(
        search_type=search_type, search_kwargs={"k": k}
    )

def process_query(query: str, retriever):
    """
    Processes the query using the provided retriever to retrieve relevant document chunks.

    Args:
        query (str): The query string to search for relevant documents.
        retriever: The retriever object configured to use the vector store for document retrieval.

    Returns:
        str: A string containing the formatted content and metadata of the retrieved document chunks.
    """
    # Retrieve chunks based on the query
    docs = retriever.get_relevant_documents(query)

    # Initialize an empty string to collect all outputs
    full_output = ""

    for i, doc in enumerate(docs, 1):
        chunk_output = f"-----Chunk {i}------\n"
        chunk_output += f"Content: {doc.page_content}...\n"
        chunk_output += f"Metadata {doc.metadata}\n\n"

        # Append the chunk output to the full output
        full_output += chunk_output

    return full_output

def get_groq_response(client, prompt, model="llama3-70b-8192", max_tokens=2048, temperature=0.0):
    """
    Generates a response using the provided client, model, prompt, and specified parameters.

    Args:
        client: The client object to interact with the API.
        prompt (str): The prompt to generate a response for.
        model (str, optional): The model identifier to use for generating the response. Default is "llama3-70b-8192".
        max_tokens (int, optional): The maximum number of tokens for the generated response. Default is 2048.
        temperature (float, optional): The temperature setting for the response generation. Default is 0.0.

    Returns:
        tuple: The generated response content and usage statistics.
    """
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model=model,
            max_tokens=max_tokens,
            temperature=temperature
        )
        return chat_completion.choices[0].message.content, chat_completion.usage
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

def generate_prompt_hyde(instruction, user_query, context_hyde):
    """
    Generates a prompt for HyDE by replacing placeholders in the instruction template with the user's query and context.

    Args:
        instruction (str): The template instruction containing placeholders.
        user_query (str): The user's query to be inserted into the instruction.
        context_hyde (str): The context for creating a hypothetical answer to be inserted into the instruction.

    Returns:
        str: The generated instruction with the placeholders replaced by the user's query and context.
    """
    instruction = instruction.replace("{USER_QUERY}", user_query)
    instruction = instruction.replace("{CONTEXT_HYDE}", context_hyde)
    return instruction

def generate_prompt_final(instruction, user_query, context_figure_table, context_rag_hyde, context_rag_general):
    """
    Generates a final prompt by replacing placeholders in the instruction template with the user's query and various contexts.

    Args:
        instruction (str): The template instruction containing placeholders.
        user_query (str): The user's query to be inserted into the instruction.
        context_figure_table (str): The context(description) related to figure and table to be inserted into the instruction.
        context_rag_hyde (str): The context retreived from RAG HyDE to be inserted into the instruction.
        context_rag_general (str): The general context retreived from RAG to be inserted into the instruction.

    Returns:
        str: The generated instruction with the placeholders replaced by the user's query and contexts.
    """
    instruction = instruction.replace("{USER_QUERY}", user_query)
    instruction = instruction.replace("{CONTEXT_FIGURE_TABLE}", context_figure_table)
    instruction = instruction.replace("{CONTEXT_RAG_HYDE}", context_rag_hyde)
    instruction = instruction.replace("{CONTEXT_RAG_GENERAL}", context_rag_general)
    return instruction

def generate_prompt_extract_query(instruction, user_query):
    """
    Generates a prompt for extracting keys from the user's query by replacing placeholders in the instruction template.

    Args:
        instruction (str): The template instruction containing a placeholder for the user's query.
        user_query (str): The user's query to be inserted into the instruction.

    Returns:
        str: The generated instruction with the placeholder replaced by the user's query.
    """
    instruction = instruction.replace("{USER_QUERY}", user_query)
    return instruction

def parse_and_convert_keys(json_string):
    """
    Parse the JSON string and convert the string values in the keys list to their appropriate types.

    Args:
    json_string (str): A JSON string representing a list of dictionaries with string values for 'thesis', 'figure', and 'table'.

    Returns:
    list: A list of dictionaries with 'thesis' as int, and 'figure' and 'table' as int or None.
    """
    try:
        keys = json.loads(json_string)
        if not keys:
            return []

        converted_keys = []
        for key in keys:
            converted_key = {
                "thesis": int(key["thesis"]) if key["thesis"] else None,
                "figure": int(key["figure"]) if key["figure"] else None,
                "table": int(key["table"]) if key["table"] else None
            }
            converted_keys.append(converted_key)
        return converted_keys
    except json.JSONDecodeError as e:
        print(f"JSON decoding error: {e}")
        return []
    except KeyError as e:
        print(f"Missing key in JSON data: {e}")
        return []
    except ValueError as e:
        print(f"Value error: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []

def extract_descriptions(df, keys):
    """
    Extract and format descriptions from the dataframe based on the provided keys.

    Args:
    df (DataFrame): The dataframe containing thesis, figure, table, and description data.
    keys (list): A list of dictionaries with 'thesis' as int, and 'figure' and 'table' as int or None.

    Returns:
    list: A list of formatted descriptions corresponding to the provided keys.
    """
    formatted_descriptions = []

    for key in keys:
        thesis_num = key["thesis"]
        figure_num = key["figure"]
        table_num = key["table"]

        if figure_num is not None:
            description = df[(df["thesis_num"] == thesis_num) & (df["figure_num"] == figure_num)]["description"].values
            prefix = f"thesis{thesis_num} figure{figure_num} description: "
        elif table_num is not None:
            description = df[(df["thesis_num"] == thesis_num) & (df["table_num"] == table_num)]["description"].values
            prefix = f"thesis{thesis_num} table{table_num} description: "
        else:
            description = []
            prefix = ""

        if len(description) > 0:
            formatted_descriptions.append(prefix + description[0])
        else:
            formatted_descriptions.append(prefix + "Description not found")

    return formatted_descriptions

def extract_thesis_numbers(converted_keys):
    """
    Extracts the thesis numbers from a list of dictionaries.

    Args:
    converted_keys (list): A list of dictionaries with 'thesis', 'figure', and 'table' keys.

    Returns:
    list: A list of thesis numbers.
    """
    try:
        thesis_numbers = [item['thesis'] for item in converted_keys]
        return thesis_numbers
    except Exception as e:
        # print(f"An error occurred while extracting thesis numbers: {e}")
        return []

def get_descriptions_for_thesis_summary(thesis_numbers, table_summary):
    """
    Retrieves the descriptions for the given thesis numbers from the table_summary DataFrame.

    Args:
    thesis_numbers (list): A list of thesis numbers.
    table_summary (pd.DataFrame): The DataFrame containing thesis numbers and their descriptions.

    Returns:
    list: A list of descriptions corresponding to the thesis numbers, formatted to indicate which thesis each description belongs to.
    """
    try:
        result = []
        for thesis_num in thesis_numbers:
            description = table_summary.loc[table_summary['thesis_num'] == thesis_num, 'description'].values[0]
            result.append(f"Summary description for thesis {thesis_num}: '{description}'")
        return result
    except Exception as e:
        # print(f"An error occurred while retrieving descriptions: {e}")
        return []

In [50]:
# Prompts(Instructions)

instruction_hyde = """
### Instructions ###
You are an expert in scientific academic papers. Your task is to answer to "Users' query" below.　If the information in the "Context" below seems relevant to "Users' query", please refer to it.

### User’s query ###
{USER_QUERY}

### Context ###
{CONTEXT_HYDE}

### Output ###
"""

instruction_final = """
### Instructions ###
You are an expert in scientific academic papers. Your task is to answer to "Users' query" below.
If the information in the "Figure/Table Context" and "Text Context" below seem relevant to "Users' query", please refer to them.
"Text Context" includes several chunks from different parts of an academic paper. "Figure/Table Context" includes the descriptions related to figures or tables in an academic paper.
Please refer only to the relevant contexts for your response. There is no need to include unrelated context in your response.
If the user asks about a specific figure or table and the information is contained in the Figure/Table Context, please ensure that this information is included in your response.
If you determine that the previous conversation history is relevant, please also refer to that information to answer the user's query.
If the conversation is continuing from the previous session and no additional information is needed, you may refer to the previous conversation history and might not need to use the contexts below. (e.g., User's query: Please make your response brief).
If the contexts and the previous conversation history do not contain the necessary information and it is difficult to answer even with general knowledge and previous context, please respond with 'The information provided is insufficient to answer your question.　Could you please clarify your question?'.

##### User’s query #####
{USER_QUERY}


##### Figure/Table Context #####
{CONTEXT_FIGURE_TABLE}

##### Text Context #####
{CONTEXT_RAG_HYDE}

{CONTEXT_RAG_GENERAL}


##### Output #####
"""

instruction_extract_query = """
### Instructions ###
You are an NLP engineer. Your task is to extract the "numbers" from the user's query below.
The "numbers" mean which academic paper the user is referring to, 2) which figure the user is referring to, and 3) which table the user is referring to.
There may be cases where all, some, or none of these are specified. Enter the number only for the specified fields, and return an empty string "" for fields that are not specified. Interpret "figure" for terms such as "Chart," "Diagram," or "Image."
Please provide your response as a list of objects, each containing thesis, figure, and table.　Please provide your response strictly in the specified format, without including any additional text for formatting. I will use your response directly.
If it is unclear which thesis, figure, or table is being referred to, it is okay to return an empty string. Please do not make any assumptions.

### Output Format ###
Format: a list of objects

### Example user's query1 ###
What is the main hypothesis or research question addressed in the first academic article?

### Example Output1 ###
[
  {
  "thesis": "1",
  "figure": "",
  "table": ""
  }
]

### Example user's query2 ###
Summarize the methodology used in the third academic article. Highlight any unique approaches or techniques employed.

### Example Output2 ###
[
  {
  "thesis": "3",
  "figure": "",
  "table": ""
  }
]


### Example user's query3 ###
Q. From the images and figures in the second article, describe the trend shown in Figure 2. What does it indicate about the research findings?

### Example Output3 ###
[
  {
  "thesis": "2",
  "figure": "2",
  "table": ""
  }
]

### Example user's query4 ###
Q. What can be understood from Image 3 in the third paper?

### Example Output4 ###
[
  {
  "thesis": "3",
  "figure": "3",
  "table": ""
  }
]

### Example user's query4 ###
Q. Please explain Figure 3 and Table 2 of the second academic paper. What do these indicate about the research findings?

### Example Output4 ###
[
  {
  "thesis": "2",
  "figure": "3",
  "table": ""
  },
  {
  "thesis": "2",
  "figure": "",
  "table": "2"
  }
]

### Example user's query5 ###
Q. Please compare table 3 and chart 4 from the second and third theses, respectively.

### Example Output5 ###
[
  {
  "thesis": "2",
  "figure": "",
  "table": "3"
  },
  {
  "thesis": "3",
  "figure": "4",
  "table": ""
  }
]

### Example user's query6 ###
Do you like an apple?

### Example Output6 ###
[
  {
  "thesis": "",
  "figure": "",
  "table": ""
  }
]


### User’s query ###
{USER_QUERY}

### Output ###
"""



In [51]:
# Main - Prepare data/databases

## Load PDF
# Load Sample PDF　- Replace with your path
pdf_path = "attention.pdf"
# text to document
text = extract_text(pdf_path)
doc = Document(page_content=text)
docs = [doc]

## Chunk
# Split the documents into chunks
doc_splits = split_documents_into_chunks(docs)
# Add chunk number to metadata
doc_splits = add_chunk_numbers_to_metadata(doc_splits)

## Embedding
# SciBERT(Allen Institute for AI) - for academic(science) paper including computer science - maximum 512 tokens
embeddings = HuggingFaceEmbeddings(model_name="allenai/scibert_scivocab_uncased")

## Vector Store
# Configure FAISS as Vector Store
# !!!!!!!!!!!!!!!!!!!! Need to keep the document number - test includes only 1 document !!!!!!!!!!!!!!!!!!!!!
vector_db = configure_faiss_vector_store(doc_splits, embeddings)
 # Expose index to the retriever
retriever = create_retriever(vector_db)

## Load prepared tables
# Load discription table(sample)
table_figure_table = [
    {"thesis_num": 1, "figure_num": 1, "table_num": None, "description": "The Transformer model architecture consists of an Encoder and a Decoder. The Encoder includes Input Embedding to convert tokens into dense vectors, Positional Encoding to add position information, Multi-Head Attention to focus on different input parts, Add & Norm for residual connections and normalization, Feed Forward to apply a position-wise feed-forward neural network, and another Add & Norm. The Decoder involves Output Embedding to convert previous output tokens, Positional Encoding, Masked Multi-Head Attention to focus on previous outputs with masking, Add & Norm, Multi-Head Attention to attend to encoder outputs, another Add & Norm, Feed Forward, and another Add & Norm. The final layers include a Linear Layer to transform decoder output and Softmax to produce a probability distribution over output tokens. This architecture uses attention mechanisms to handle long-range dependencies in sequences, making it effective for tasks like machine translation and text summarization."},
    {"thesis_num": 1, "figure_num": 2, "table_num": None, "description": "This is description for figure2 in thesis 1"},
    {"thesis_num": 1, "figure_num": 3, "table_num": None, "description": "This is description for figure3 in thesis 1"},
    {"thesis_num": 1, "figure_num": 4, "table_num": None, "description": "This is description for figure4 in thesis 1"},
    {"thesis_num": 2, "figure_num": 1, "table_num": None, "description": "This is description for figure1 in thesis 2"},
    {"thesis_num": 2, "figure_num": 2, "table_num": None, "description": "This is description for figure2 in thesis 2"},
    {"thesis_num": 2, "figure_num": 3, "table_num": None, "description": "This is description for figure3 in thesis 2"},
    {"thesis_num": 2, "figure_num": None, "table_num": 1, "description": "This is description for table1 in thesis 2"},
    {"thesis_num": 2, "figure_num": None, "table_num": 2, "description": "This is description for table2 in thesis 2"},
    {"thesis_num": 2, "figure_num": None, "table_num": 3, "description": "This is description for table3 in thesis 2"},
    {"thesis_num": 3, "figure_num": 1, "table_num": None, "description": "This is description for figure1 in thesis 3"},
    {"thesis_num": 3, "figure_num": 2, "table_num": None, "description": "This is description for figure2 in thesis 3"},
    {"thesis_num": 3, "figure_num": None, "table_num": 1, "description": "This is description for table1 in thesis 3"},
    {"thesis_num": 3, "figure_num": None, "table_num": 2, "description": "This is description for table2 in thesis 3"},
    {"thesis_num": 3, "figure_num": None, "table_num": 3, "description": "This is description for table3 in thesis 3"},
    {"thesis_num": 4, "figure_num": 1, "table_num": None, "description": "This is description for figure1 in thesis 4"},
    {"thesis_num": 4, "figure_num": 2, "table_num": None, "description": "This is description for figure2 in thesis 4"},
    {"thesis_num": 4, "figure_num": 3, "table_num": None, "description": "This is description for figure3 in thesis 4"},
    {"thesis_num": 4, "figure_num": None, "table_num": 1, "description": "This is description for table1 in thesis 4"},
    {"thesis_num": 4, "figure_num": None, "table_num": 2, "description": "This is description for table2 in thesis 4"},
    {"thesis_num": 4, "figure_num": None, "table_num": 3, "description": "This is description for table3 in thesis 4"}
]

table_figure_table = pd.DataFrame(table_figure_table)

# Load table with thesis_num and description
table_summary = pd.DataFrame([
    {"thesis_num": 1, "description": "The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English- to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data."},
    {"thesis_num": 2, "description": "This is summary description in thesis 2"},
    {"thesis_num": 3, "description": "This is summary description in thesis 3"},
    {"thesis_num": 4, "description": "This is summary description in thesis 4"}
])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [52]:
# LLM for the main flow
client_main = Groq(
    api_key="YOUR_API_KEY_1",
)
#  LLM for keys(thesis/figure/table) extaction
client_extract = Groq(
    api_key="YOUR_API_KEY_2",
)
# LLM for HyDE
client_hyde = Groq(
    api_key="YOUR_API_KEY_3",
)


In [53]:
# Main - Answering questions

def chat_main(user_query):

  ## 1. extract keys from user's query and find figure/table description and summary description ##
  # generate prompt to extract keys from user's query
  prompt_extract_query = generate_prompt_extract_query(instruction_extract_query, user_query)
  # get keys from Extraction LLM
  response_keys = get_groq_response(client_extract, prompt_extract_query)
  # parse keys
  keys = parse_and_convert_keys(response_keys[0])
  # extract figure/table descriptions
  descriptions_figure_table = extract_descriptions(table_figure_table, keys)

  # extract thesis numbers from keys
  keys_thesis = extract_thesis_numbers(keys)
  # get summary descriptions
  descriptions_summary = get_descriptions_for_thesis_summary(keys_thesis, table_summary)


  ## 2. get context for HyDE and general RAG ##
  # !!!!!!!!!!!!!!!!!!! filter vectorDB by using thesis number(keys_thesis) - process_query!!!!!!!!!!!!!!!!!!!
  # ADD_CODE_HERE

  # add summary of the thesis as a context for HyDE
  context_hyde = descriptions_summary
  # create prompt for HyDE
  prompt_hyde = generate_prompt_hyde(instruction_hyde, user_query, str(context_hyde))
  # get a hypothetical answer from HyDE LLM
  response_hyde = get_groq_response(client_hyde, prompt_hyde)
  # create contexts - RAG(Hyde)
  context_rag_hyde = process_query(response_hyde[0], retriever)
  # create contexts - RAG(General)
  context_rag_general = process_query(user_query, retriever)


  ## 3. get a final response ##
  # create prompt for a final response
  prompt_final = generate_prompt_final(instruction_final, user_query, str(descriptions_figure_table), context_rag_hyde, context_rag_general)
  # get final response from main LLM
  response_final = get_groq_response(client_main, prompt_final)

  return response_final

In [54]:
# get response - figure test
user_query = 'Describe the Figure 1 in the first paper.'
response = chat_main(user_query)
print(response[0])

Figure 1 in the first paper illustrates the Transformer model architecture, which consists of an Encoder and a Decoder. The Encoder includes Input Embedding, Positional Encoding, Multi-Head Attention, Add & Norm, Feed Forward, and another Add & Norm. The Decoder involves Output Embedding, Positional Encoding, Masked Multi-Head Attention, Add & Norm, Multi-Head Attention, another Add & Norm, Feed Forward, and another Add & Norm. The final layers include a Linear Layer and Softmax to produce a probability distribution over output tokens. This architecture uses attention mechanisms to handle long-range dependencies in sequences, making it effective for tasks like machine translation and text summarization.


In [55]:
# get response - summary test
user_query = 'Identify key findings from the first academic article'
response = chat_main(user_query)
print(response[0])

Based on the provided Text Context, the key findings from the first academic article are:

1. The proposed Transformer model achieves a new state-of-the-art BLEU score of 28.4 on the WMT 2014 English-to-German translation task, outperforming existing models, including ensembles, by over 2 BLEU.
2. The Transformer model also establishes a new single-model state-of-the-art BLEU score of 41.8 on the WMT 2014 English-to-French translation task, after training for 3.5 days on eight GPUs, which is a small fraction of the training costs of the best models from the literature.
3. The Transformer model generalizes well to other tasks, such as English constituency parsing, both with large and limited training data.
4. The model can be trained significantly faster than architectures based on recurrent or convolutional layers, achieving a new state of the art on both WMT 2014 English-to-German and WMT 2014 English-to-French translation tasks.
5. The attention mechanism in the Transformer model all

In [56]:
# get response - conversation history test
user_query = 'Please make the findings brief'
response = chat_main(user_query)
print(response[0])

Based on the provided context, the findings can be briefly summarized as follows:

* The paper explores the use of self-attention mechanisms in sequence transduction models, highlighting their potential benefits in terms of interpretability and performance.
* The authors demonstrate that self-attention layers can learn to perform different tasks and exhibit behavior related to the syntactic and semantic structure of sentences.
* The paper also presents results from experiments comparing self-attention layers to recurrent and convolutional layers, showing that self-attention can be a viable alternative.
* Additionally, the authors discuss the importance of considering the total computational complexity per layer and the amount of computation that can be parallelized.

Overall, the paper presents a novel approach to sequence transduction using self-attention mechanisms and demonstrates its potential benefits and advantages.
